<a href="https://colab.research.google.com/github/guilhermevaz8/RL_Taxi_Problem/blob/main/Hyperparam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install gymnasium comet_ml stable-baselines3 sb3_contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.8/586.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.7.0
    Uninstalling websocket-client-1.7.0:
      Successfully uninstalled websocket-client-

In [2]:
import comet_ml

comet_ml.init(project_name="comet-examples-gymnasium-notebook")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [4]:
pip install ray

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 11.8 MB/s eta 0:00:00


In [ ]:
import gym
import numpy as np
from ray import tune
from comet_ml import Experiment
from stable_baselines3.common.callbacks import EvalCallback
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

def mask_fn(env):
    # Implement your action mask function here
    return env.unwrapped.action_mask(env.unwrapped.s)

def train_maskable_ppo(config):
    env = gym.make("Taxi-v3")
    env = ActionMasker(env, mask_fn)

    model = MaskablePPO(
        MaskableActorCriticPolicy,
        env,
        learning_rate=config['learning_rate'],
        ent_coef=config['ent_coef'],
        verbose=0
    )

    # Treina o modelo por um número fixo de iterações
    model.learn(total_timesteps=5000000)

    return model

if __name__ == "__main__":
    # Realiza a otimização de hiperparâmetros com o Ray Tune
    analysis = tune.run(
        train_maskable_ppo,
        config={
            'learning_rate': tune.loguniform(1e-5, 1e-2),
            'ent_coef': tune.uniform(0.0, 0.2)
        },
        num_samples=50,
        verbose=1
    )

    # Obtém a melhor configuração
    best_config = analysis.get_best_config(metric="mean_reward", mode="max")

    # Treina o MaskablePPO com a melhor configuração
    best_model = train_maskable_ppo(best_config)

    # Avalia o melhor modelo e salva os resultados no Comet
    eval_env = gym.make("Taxi-v3")
    eval_env = ActionMasker(eval_env, mask_fn)

    mean_reward, _ = best_model.evaluate(eval_env, n_eval_episodes=5)

    experiment = Experiment()
    experiment.log_metric("final_best_mean_reward", mean_reward)
    experiment.end()


2023-12-15 12:15:36,246	INFO tune.py:595 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-12-15 12:15:36,299	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


+---------------------------------------------------------------------------+
| Configuration for experiment     train_maskable_ppo_2023-12-15_12-15-36   |
+---------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                    |
| Scheduler                        FIFOScheduler                            |
| Number of trials                 50                                       |
+---------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_maskable_ppo_2023-12-15_12-15-36

Trial status: 16 PENDING
Current time: 2023-12-15 12:15:36. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+--------------------------------------------------------------------------+
| Trial name                       status       learning_rate     ent_coef |
+--------------------------------------------------------------------------+
| train_ma